In [12]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [3]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [4]:
sing_cubics_new = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [5]:
sing_cubics_new_factored = sing_cubics_new.factor()
sing_cubics_new_factored

(-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
line = Line([y, z])

In [7]:
lines = find_all_lines_on_cubic_surface(line, cubic_new)

In [8]:
cl_lines = classify_lines_on_cubic_surface(lines)

In [9]:
all_L_sets = find_all_L_sets(cl_lines)

In [10]:
tritangent_planes = find_all_tritangent_planes(cl_lines)

In [11]:
0 in [pl.conditions.subs({b:-(c*c+e*f)/c }) for pl in tritangent_planes if pl.labels != ['E1', 'G4', 'F14']]

False

In [12]:
SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )
SE1

-x^2*z*c^2 - x*z^2*c^2 - x*y*t*c^2 + y*z*t*c^2 + 2*y*t^2*c^2 + x*y*z*c*d - x*z^2*c*d + y^2*t*c*d - y*z*t*c*d - x^2*z*c*e + x*y*z*c*e + x*y*t*c*e - y^2*t*c*e - x^2*z*c*f + x*y*z*c*f + x*y*t*c*f - y^2*t*c*f + 2*x^2*y*e*f - 2*x*y^2*e*f - x^2*z*e*f + x*z^2*e*f - x*y*t*e*f + y*z*t*e*f

In [13]:
0 in [pl.conditions.subs({b:-(c*c+e*f)/c, d:(c*c+e*f)/c }) for pl in tritangent_planes if pl.labels not in [ ['E1', 'G4', 'F14'], ['E2', 'G4', 'F24']]]

False

In [14]:
SE2 = P(SE1.subs({d:(c*c+e*f)/c}) )
SE2

-x^2*z*c^2 + x*y*z*c^2 - 2*x*z^2*c^2 - x*y*t*c^2 + y^2*t*c^2 + 2*y*t^2*c^2 - x^2*z*c*e + x*y*z*c*e + x*y*t*c*e - y^2*t*c*e - x^2*z*c*f + x*y*z*c*f + x*y*t*c*f - y^2*t*c*f + 2*x^2*y*e*f - 2*x*y^2*e*f - x^2*z*e*f + x*y*z*e*f - x*y*t*e*f + y^2*t*e*f

In [15]:
lab = []
for pl in tritangent_planes:
    if pl.conditions.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)  }) == 0:
        lab.append(pl.labels)

In [16]:
lab

[['E1', 'G4', 'F14'], ['E2', 'G3', 'F23'], ['F13', 'F24', 'F56']]

In [17]:
SE3 = P(2*SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}) )
SE3

-2*x^2*z*c^2 - x*y*z*c^2 - x*z^2*c^2 - 2*x*y*t*c^2 - y^2*t*c^2 + 3*y*z*t*c^2 + 4*y*t^2*c^2 - 2*x^2*z*c*e + 3*x*y*z*c*e - x*z^2*c*e + 2*x*y*t*c*e - y^2*t*c*e - y*z*t*c*e - 2*x^2*z*c*f + 3*x*y*z*c*f - x*z^2*c*f + 2*x*y*t*c*f - y^2*t*c*f - y*z*t*c*f + 4*x^2*y*e*f - 4*x*y^2*e*f - 2*x^2*z*e*f + 3*x*y*z*e*f - x*z^2*e*f - 2*x*y*t*e*f + 3*y^2*t*e*f - y*z*t*e*f

In [18]:
sost = {b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}
[P(pl.conditions.subs(sost)*pl.conditions.subs(sost).denominator()).factor() for pl in tritangent_planes if pl.conditions.subs(sost) !=0]

[(1/4) * (c + f)^2 * (c + e)^2 * (5*c^2 - c*e - c*f + e*f),
 -3*c^2 - c*e - c*f + e*f,
 (-1) * c * (c^2 + 3*c*e - c*f + e*f),
 (-1) * c * (c^2 - c*e + 3*c*f + e*f),
 (1/4) * (-c + f) * (-c + e) * (c + f) * (c + e) * (5*c^2 + c*e + c*f + e*f),
 (-1/2) * (-3*c^2 + c*e + c*f + e*f),
 (-1/2) * (c + e) * (c^2 + c*e - 3*c*f + e*f),
 (-1/2) * (c + f) * (c^2 - 3*c*e + c*f + e*f),
 (c + f) * (c + e) * (5*c^2 - c*e - c*f + e*f),
 (-8) * (c + f) * (c + e) * (c^2 + e*f),
 (-1/2) * (5*c^2 + c*e + c*f + e*f),
 (1/2) * (c + f) * (3*c^2 + e^2),
 (1/2) * (c + e) * (3*c^2 + f^2),
 (2) * (c + f) * (c + e) * c^2 * (c^2 + e*f),
 (1/4) * (-c + f) * (-c + e) * (c + f) * (c + e) * (5*c^2 + c*e + c*f + e*f),
 (-1/2) * (c + f) * (c + e) * (5*c^2 - c*e - c*f + e*f),
 (-1/2) * (-c + e) * (c + f) * (c + e) * (3*c^2 + f^2),
 (1/2) * (-c + f) * (c + f) * (c + e) * (3*c^2 + e^2),
 (1/2) * (-c + e) * c * (c + f) * (c + e) * (3*c^2 + f^2),
 (1/4) * (-c + f) * (c + e) * (c + f)^2 * (3*c^2 + e^2),
 (-1/2) * (-c + e) * (c

In [19]:
SE4 = -P((c+e) * SE3.subs({f:c*(3*c-e)/(c+e)}) / (4*c))
SE4

2*x^2*z*c^2 - 2*x*y*z*c^2 + x*z^2*c^2 - x*y*t*c^2 + y^2*t*c^2 - y*t^2*c^2 - 3*x^2*y*c*e + 3*x*y^2*c*e + 2*x^2*z*c*e - 2*x*y*z*c*e + x*z^2*c*e + 2*x*y*t*c*e - 2*y^2*t*c*e - y*t^2*c*e + x^2*y*e^2 - x*y^2*e^2 - x*y*t*e^2 + y^2*t*e^2

In [20]:
SE6 = P((c+e)*SE3.subs({f: -c*(5*c+e)/(c+e)})/(4*c))
SE6

2*x^2*z*c^2 - 4*x*y*z*c^2 + x*z^2*c^2 - 3*x*y*t*c^2 + y^2*t*c^2 + 2*y*z*t*c^2 + y*t^2*c^2 - 5*x^2*y*c*e + 5*x*y^2*c*e + 2*x^2*z*c*e - 4*x*y*z*c*e + x*z^2*c*e + 2*x*y*t*c*e - 4*y^2*t*c*e + 2*y*z*t*c*e + y*t^2*c*e - x^2*y*e^2 + x*y^2*e^2 + x*y*t*e^2 - y^2*t*e^2

In [21]:
#SE9 = (SE3.subs({e : alpha*c})/c).factor().expand()
#SE9

In [22]:
#SE10 = (SE4.subs({e : (2-sqrt(5))*c})).factor()
#SE10

In [23]:
#SE18 = (SE4.subs({e : alpha*c})/(c^2 * (alpha+1)))
#SE18

In [24]:
pl = tritangent_planes[26]
mcd = gcd(pl.conditions)
other_cond = [P(el/mcd) for el in pl.conditions]

TypeError: 'int' object is not iterable

In [ ]:
P.ideal(other_cond).saturation(c)[0].radical().primary_decomposition()

In [ ]:
#10, 11, 18, 19,23,24,28,29,37,38,40,41,43,44

In [ ]:
for i in range(45):
    print(i)
    if i == 26 or i == 21:
        continue
    prim_dec = P.ideal(tritangent_planes[i].conditions).radical().primary_decomposition()
    if len(prim_dec) > 1:
        prim_dec = P.ideal(tritangent_planes[i].conditions).saturation(c)[0].radical().primary_decomposition()
    #10, 11, 18, 19,23,24,28,29,37,38,40,41,43,44
    if i in [10, 11, 18, 19,24,29,37,38,40,41,43,44]:
        print(c*prim_dec[0].gens()[0] == gcd(tritangent_planes[i].conditions))
    elif i in [23,28]:
        print(-c*prim_dec[0].gens()[0] == gcd(tritangent_planes[i].conditions))        
    else:
        print(prim_dec[0].gens()[0] == gcd(tritangent_planes[i].conditions))        
    print()
    sleep(1)

In [ ]:
#10, 11, 15, 18, 19,22,23,27,28,37,38,40,41,43,44